In [91]:
import rasterio
import numpy as np
import fiona
import pandas as pd
import fiona
import time
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
from rasterio import windows as win

In [92]:
nlcd_lc_Address_2011=r'G:\NationalLayer\NLCD_Products\2011\nlcd_2011_landcover_2011_edition_2014_10_10\nlcd_2011_landcover_2011_edition_2014_10_10\nlcd_2011_landcover_2011_edition_2014_10_10_projected.img'
nlcd_lc_Address_2016=r'G:\NationalLayer\NLCD_Products\2016\NLCD_2016_Land_Cover_L48_20190424\NLCD_2016_Land_Cover_L48_20190424_projected.img'
ecoAcAreasShapefile=r'G:\NationalLayer\Census\UrbanAreas\nhgis0054_shape_Places_2010_withPop_hsng\US_place_2010_Projected_50k_pop.shp' 
ecoAcAreasRaster=r'G:\NationalLayer\Census\UrbanAreas\nhgis0054_shape_Places_2010_withPop_hsng\Places_50k_30m_raster.img'

# set the fields to get name and IDs
nameField='NAMELSAD10'
id_field='Pl_Uniq_id'


In [103]:
# get the list of codes and the list of names
lcCodes=[0,11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90, 95]
lcNames=['','Open Water','Perennial Ice/Snow','Developed Open Sapce','Developed Low Intensity',
         'Developed Medium Intensity','Developed High Intensity','Barren Land','Deciduous Forest','Evergreen Forest',
         'Mixed Forest','Dwarf','Shrub','Grassland','Sedge/Herb','Lichens','Moss','Pasture/hay','Cultivated Crops',
        'Woody Wetlands','Emergent Herbaceous Wetlands']

def creatDictionary (lstCodes,lstNames):
    dictNames={}
    for i in range(0,len(lstCodes)):
        dictNames[lstCodes[i]]=lstNames[i]
    return dictNames

# create a dictionary for landcover codes and names
dicLcCodes=creatDictionary(lcCodes,lcNames)
# create a dictionary for landuse codes and names

def landCoverCodeConvert(codeNumb):
    return (dicLcCodes[codeNumb])

def nameCodeCol (dfArg,codeCol,nameCol,dictCodeNames):
    dfArg[nameCol]=''
    lstUniques=(dfArg[codeCol]).unique()
    for code in lstUniques:
        dfArg.loc[(dfArg[codeCol]==code),nameCol]=dictCodeNames[code]

In [104]:
def findWindow (shapeBound,mainRasterBnd,mainRasterCellSize):
    startRow = int((mainRasterBnd[3] - shapeBound[3])/mainRasterCellSize)
    endRow   = int((shapeBound[3] - shapeBound[1])/mainRasterCellSize)+1+startRow
    startCol = int((shapeBound[0] - mainRasterBnd[0])/mainRasterCellSize)
    endCol   = int((shapeBound[2] - shapeBound[0])/mainRasterCellSize)+1+startCol
    return (startRow,endRow,startCol,endCol)

In [96]:
lstDFsLandcover=[]

for pol in fiona.open(ecoAcAreasShapefile):
    #try:
    #for pol in fiona.open(ecoAcAreasShapefile):
    eaa_name=(pol['properties'][nameField])
    eaa_state=(pol['properties'])['StateName']
    GISjoins=(pol['properties']['GISJOIN'])
    eaa_id=(pol['properties'][id_field])
    cso=(pol['properties']['CSO_StormW'])
    Valid_ST=(pol['properties']['Valid_ST'])
    # kbtu per sq feet for residential use per year
    kbtu=(pol['properties']['KBTU'])
    poly_Ycoor=(pol['properties']['Y_coor'])
    poly=(shape(pol['geometry']))
    
    with rasterio.open(nlcd_lc_Address_2016) as rst_nlcd_tc:
        kwds = rst_nlcd_tc.meta.copy()
        mainRasterBnd=rst_nlcd_tc.bounds
        cellSize= kwds['transform'][0]

    polyBound = poly.bounds

    # create a window parameter tuple.   
    winProcessing=findWindow(polyBound,mainRasterBnd,cellSize)
    #(row_start, row_stop), (col_start, col_stop)
    window_use=((winProcessing[0],winProcessing[1]),(winProcessing[2],winProcessing[3]))

    # set the cells that do not have the city id as np.nan. This way we are getting cells insdie the boundary only.
    with rasterio.open(ecoAcAreasRaster) as src:
        eaaAr=src.read(1, window=window_use)
        eaaNodata=src.nodata
        arrayShapes=eaaAr.shape
        eaaAr=eaaAr.flatten()
    
    with rasterio.open(nlcd_lc_Address_2016) as src:
        nlcd_lc=src.read(1, window=window_use)
        nlcd_lc_nodata=src.nodata
        nlcd_lc=nlcd_lc.astype('float')
        #nlcd_lc[eaaAr!=eaa_id]=np.nan
        nlcd_lc=nlcd_lc.flatten()
    df=pd.DataFrame(nlcd_lc,columns=['LC'])
    dfAgrLandcover=df.groupby(['LC']).agg({'LC': ['count']})
    dfAgrLandcover.columns = dfAgrLandcover.columns.droplevel(1)

    dfAgrLandcover['city']=eaa_name
    dfAgrLandcover['eaa_id'] =eaa_id
    dfAgrLandcover['GISJOIN'] = GISjoins
    dfAgrLandcover['CSO'] = cso
    dfAgrLandcover['Valid_ST'] = Valid_ST

    dfAgrLandcover['count'] =(dfAgrLandcover['LC']).astype('int64')
    dfAgrLandcover['LC_AreaM2']=(dfAgrLandcover['count']*900)
    dfAgrLandcover['LC'] = (dfAgrLandcover.index).astype('int8')
    lstDFsLandcover.append(dfAgrLandcover)

In [105]:
allDfsLandcover = pd.concat(lstDFsLandcover)
nameCodeCol(allDfsLandcover,'LC','LC_name',dicLcCodes)

In [106]:
allDfsLandcover.sample(10)

,LC,city,eaa_id,GISJOIN,CSO,Valid_ST,count,LC_AreaM2,LC_name
LC,,,,,,,,,
21.0,21,East Orange city,555,G3419390,0,1,3024,2721600,Developed Open Sapce
95.0,95,Shawnee city,426,G2064500,0,1,170,153000,Emergent Herbaceous Wetlands
82.0,82,Oak Lawn village,384,G1754820,0,1,17,15300,Cultivated Crops
90.0,90,Cheyenne city,762,G5613900,0,1,1020,918000,Woody Wetlands
22.0,22,El Cajon city,65,G0621712,0,1,20033,18029700,Developed Low Intensity
90.0,90,Hayward city,166,G0633000,0,1,214,192600,Woody Wetlands
42.0,42,Springdale city,246,G0566080,0,1,106,95400,Evergreen Forest
22.0,22,Syracuse city,573,G3673000,1,1,36056,32450400,Developed Low Intensity
52.0,52,Lawrence city,432,G2038900,0,1,139,125100,Shrub


In [107]:
folderAddress = r'D:\OneDriveSyncFolder\OneDrive - The University of Colorado Denver\Ecosystem Service Accounting Project\Papers\Urban EA\Results\2020Version'
allDfsLandcover.to_csv(folderAddress +    r'\ES_extent_2016.csv')